# Characterize dynamics of HFCLKAUDIO

Use an oscilloscope to capture the dynamics of the HFCLKAUDIO PLL when the frequency is changed. The nRF53 was configured with I2S MCLK bypass mode to output the raw HFCLKAUDIO to a pin. It would periodically step the frequency up and down by a 1000 steps. This is a much larger step than you are supposed to use, but it seems to work fine, although the dynamics may vary with step size if the PLL is not perfectly linear. Additionally, toggle a GPIO each time the frequency is changed to provide a trigger input for the scope. I used a 100 MHz / 1 GSa/s Rigol DS1102E to capture the data, so this limited my ability to observe smaller step changes. Even at 1000 steps, I had to use this notebook to combine many measurements and filter out the noise.

I used `hfclkaudio_capture.py` to automate data collection from the scope. That script does most of the configuration of the scope and will repeatedly capture step waveforms.

Misc notes:
* Might want to increase pin drive strength
* I accidentally used a 1X probe to collect my data, so this code assumes rather slow transitions. In particular, the quadratic curve fitting might not work well or be necessary with a nice sharp edge. On the other hand, the gradual transition may increase the effective time resolution of the scope.

In the end, this whole exercise was pretty useless because the dynamics damp out very quickly relative to the controller period and can therefore be ignored.

In [ ]:
%matplotlib widget
import glob
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt

plt.close("all")

In [ ]:
# Load data captured from scope
def load_data(file):
    data = np.load(file)

    time = data[0, :]
    volts = data[1, :]

    # Remove data before trigger
    after_trigger = time >= 0
    time = time[after_trigger]
    volts = volts[after_trigger]

    return time, volts


# Split data at each transition point (local min/max)
def split_transitions(time, volts):
    peaks, _ = scipy.signal.find_peaks(volts, prominence=0.25, distance=50)
    troughs, _ = scipy.signal.find_peaks(-volts, prominence=0.25, distance=50)
    splits = np.concatenate((peaks, troughs))
    splits.sort()

    time_segments = np.split(time, splits)
    volts_segments = np.split(volts, splits)

    return time_segments, volts_segments


# Clip top and bottom of each transition and then fit a quadratic to the remaining data. Calculate the intersection point of this quadratic with a threshold line.
def crossing_time(t, v, thresh, clip=0.1):
    v_min = np.min(v)
    v_max = np.max(v)
    v_pkpk = v_max - v_min
    clipped = (v > (v_min + v_pkpk * clip)) & (v < (v_max - v_pkpk * clip))
    clip_index = np.where(np.diff(np.hstack(([False], clipped, [False]))))[0]
    t = t[clip_index[0] : clip_index[1]]
    v = v[clip_index[0] : clip_index[1]]

    a, b, c = np.polyfit(t, v, 2)
    v_fit = a * t * t + b * t + c

    pm_term = np.sqrt(b * b - 4 * a * (c - thresh))
    if b > 0:
        t_thresh = (-b + pm_term) / (2 * a)
    else:
        t_thresh = (-b - pm_term) / (2 * a)

    return t_thresh


# Use the threshold crossings to measure the period of each cycle
def cycle_periods(time_segments, volts_segments):
    threshold = 3.3 / 2
    crossings = []
    # Skip first and last segment because they are usually incomplete
    for time_segment, volts_segment in zip(
        time_segments[1:-1:2], volts_segments[1:-1:2]
    ):
        crossings.append(crossing_time(time_segment, volts_segment, threshold))
    crossings = np.array(crossings)
    return crossings[:-1], np.diff(crossings)

In [ ]:
time, volts = load_data("../../data/hclkaudio_step/step_1000_33.npy")

fig, ax = plt.subplots()
ax.plot(time, volts)

In [ ]:
files = list(glob.glob("../../data/hclkaudio_step/step_1000_*.npy"))

t = np.linspace(0, 1.4e-5, 1000)

step_up_periods = []
step_down_periods = []
for i, file in enumerate(files):
    time, volts = load_data(file)
    time_segments, volts_segments = split_transitions(time, volts)
    cycle_time, cycle_period = cycle_periods(time_segments, volts_segments)
    # Determine step direction by fitting a line and checking slope
    slope, _ = np.polyfit(cycle_time, cycle_period, 1)

    cycle_period_interp = np.interp(t, cycle_time, cycle_period)
    if slope < 0:
        step_up_periods.append(cycle_period_interp)
    else:
        step_down_periods.append(cycle_period_interp)

periods = np.mean(step_up_periods, axis=0)
freq = 1 / periods

fig, ax = plt.subplots()
ax.plot(t, freq / 1e6);

A second order model is close but not perfect. There are clearly some higher order components or non-linearities.

In [ ]:
def u_to_f(u):
    return 32e6 / 12 * (4 + u * 2**-16)


u_0 = 15309
u_step = 1000
k = 32e6 / 12 * 2**-16
f_0 = u_to_f(u_0)
f_ss = f_0 + u_step * k
f_peak = 11.3437e6

mp = (f_peak - f_ss) / (f_ss - f_0)
zeta = -np.log(mp) / np.sqrt(np.pi**2 + np.log(mp) ** 2)
omega_n = 180e3 * 2 * np.pi

# State space form
A = np.array([[0, 1, 0], [0, 0, 1], [0, -(omega_n**2), -2 * zeta * omega_n]])
B = np.array([[0], [0], [omega_n**2 * k]])
C = np.array([[0, 1, 0]])
D = np.array([[0]])
sys = scipy.signal.StateSpace(A, B, C, D)

# Alternative, transfer function form
# sys = scipy.signal.TransferFunction([k * omega_n**2], [1, 2 * zeta * omega_n, omega_n**2])

u = np.heaviside(t - 0.1e-5, 0) * u_step
tout, y, x = scipy.signal.lsim(sys, u, t)

fig, ax = plt.subplots()
ax.plot(t, y + f_0, t, freq)
ax.legend(("Model", "Data"))
display(mp)

The controller runs at 10 Hz (100 ms), while the dynamics are fully damped in <10 us. Therefore they are basically irrelevant in the discretized model.

In [ ]:
sys.to_discrete(0.1)